In [1]:
import os

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, LogisticRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, classification_report, roc_auc_score
import joblib
import pandas as pd
import numpy as np

# 回归算法 1.线性回归（正规方程版） LinearRegression()

In [ ]:
"""
    正规方程通过二次型直接求解，速度快，适用于小型数据，大数据的话时间复杂度O(m3)太大了
    正规方程版线性回归就是 y = k1x1 + k2x2 + k3x3 + b
    其中x是每个特征的参数值，k是每个特征对于结果的权重，b是偏置值，y即为最终的预测目标
    这里使用b就是为了防止该函数被迫通过原点（因为在某些情况不见得参数为0结果就是0）
    
    它内部找最小值的办法
    线性回归有一个直接求解的方法，称为 正规方程（Normal Equation）（二次型）。通过数学推导，它可以直接找到使 MSE 最小化的参数：
    
    
    评估线性回归的指标：MSE
"""

In [3]:
"""
线性回归直接预测房子价格
:return: None
"""
# 获取数据
fe_cal = fetch_california_housing(data_home='../python_ml/data')

print("获取特征值")
print(fe_cal.data.shape)
print('-' * 50)
print(fe_cal.data[0])
print("目标值")
print(fe_cal.target) #单位是10万美金
print(fe_cal.DESCR)
print('-' * 50)
print(fe_cal.feature_names) #特征列的名字

获取特征值
(20640, 8)
--------------------------------------------------
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
目标值
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/

标准化数据

In [4]:
x_train,x_test,y_train,y_test = train_test_split(fe_cal.data,fe_cal.target,test_size = 0.25,random_state = 1)
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)


开始回归算法

In [7]:
lr = LinearRegression()

lr.fit(x_train,y_train)

#y_predict返回的结果就是每个特征值对结果的影响因子的ndarray
y_predict = lr.predict(x_test)
print(y_test.shape) #目标值长度与y_predict长度相同
print(y_predict)
print("正规方程测试集里面每个房子的预测价格：", y_predict[0:10])
#下面是求测试集的损失，用均方误差，公式是(y_test-y_predict)^2/n
print("正规方程的均方误差：", mean_squared_error(y_test, y_predict))

(5160,)
[2.12391852 0.93825754 2.7088455  ... 1.24263061 2.73771901 1.75800594]
正规方程测试集里面每个房子的预测价格： [2.12391852 0.93825754 2.7088455  1.70873764 2.82954754 3.50376456
 3.0147162  1.62781292 1.74317518 2.01897806]
正规方程的均方误差： 0.5356532845422556


保存模型

In [ ]:
"""
    这个概念第一次涉及，齐用于存储之前训练模型所得的结果（说白了就是这些k值），当然之前的分类算法也可以进行保存
    也就解开了如何持续训练模型的答案（增量训练）
    
    增量训练
    不过决策树和随机森林不支持增量训练，只能将新数据和原有数据结合后重新训练
    knn和朴素贝叶斯支持增量训练，但是knn由于数据量的增加计算效率会变得非常慢，所以增量会有限
    通过   load_model.partial_fit(X_train2, y_train2) 
"""

In [ ]:
os.unlink('./tmp/test.pkl') # 删除之前的模型文件（这里只是学习如何删除模型文件）
joblib.dump(lr, "./tmp/test.pkl") # 保存模型
load_model = joblib.load('./tmp/test.pkl') #加载模型
load_model.partial_fit(X_train2, y_train2) # 增量训练（如果该算法支持增量训练）

# 回归算法 2.线性回归（梯度下降版） SGDRegressor()


In [ ]:
"""
    用梯度下降来实现线性回归，内部实现是通过迭代优化
    对比正规方程版，速度慢，适用于大型数据
"""

In [8]:
sgd = SGDRegressor()

#上面算法1已经处理完数据了，这里直接用
sgd.fit(x_train,y_train) 
y_predict = sgd.predict(x_test)
print("梯度下降的均方误差：", mean_squared_error(y_test, y_predict))

梯度下降的均方误差： 0.5392029356811116


### 模拟梯度下降的过程

In [ ]:
"""
    这段代码展示了一个典型的梯度下降优化过程，用来最小化一个简单的二次损失函数：
    1.初始化参数W和α
    2.迭代更新梯度，更新参数W
    3.损失值L（W）随着迭代次数增加而减少，最终收敛到全局最优点（若是在二次函数中，他总能结束在全局最优）
    

"""

In [9]:
w=1

# alpha为学习率，它决定了每次更新的步长。如果学习率过大，可能导致参数跳跃过多，无法收敛；如果学习率过小，收敛速度会很慢。
alpha=0.7

#损失函数 L(w) 
def loss(w):
    return 2*w**2+3*w+2
#导数，也就是L(w)对w的梯度
def dao_shu(w):
    return 4*w+3

#通过w = w - a*梯度 更新w
for i in range(30):
    
    w=w-alpha*dao_shu(w)
    print(f'w {w} 损失{loss(w)}')

w -3.8999999999999995 损失20.71999999999999
w 4.919999999999999 损失65.17279999999998
w -10.955999999999996 损失209.19987199999983
w 17.620799999999992 损失675.8475852799994
w -33.817439999999976 损失2187.786176307197
w 58.77139199999995 损失7086.4672112353155
w -107.8885055999999 损失22958.193764402422
w 192.09931007999978 损失74382.58779666382
w -347.87875814399956 损失240997.6244611907
w 624.0817646591992 损失780830.3432542577
w -1125.4471763865586 损失2529888.352143795
w 2023.7049174958051 损失8196836.300945894
w -3644.768851492449 损失26557747.65506469
w 6558.483932686408 损失86047100.4424096
w -11807.371078835533 损失278792603.4734071
w 21251.167941903957 损失903288033.2938386
w -38254.202295427116 损失2926653225.912036
w 68855.4641317688 损失9482356449.994993
w -123941.93543718383 损失30722834896.023777
w 223093.38378693088 损失99541985061.15703
w -401570.1908164755 损失322516031596.1886
w 722824.2434696557 损失1044951942369.6908
w -1301085.73824538 损失3385644293275.837
w 2341952.228841684 损失10969487510211.748
w -4215516.1

# 回归算法 3.岭回归 Ridge()

In [ ]:
"""
    岭回归是一种线性回归的扩展，主要用于解决线性回归模型中 多重共线性（即特征之间高度相关）的问题，同时可以抑制模型的过拟合。
    
    岭回归的目标是引入 L2 正则化 项，修改普通线性回归的损失函数，添加一个对回归系数大小的惩罚项。
        在普通的损失函数计算中添加一个L = L + λ*|| w ||2
            λ     ：正则化强度
        ｜｜w｜｜2 ：所有回归系数的平方和
        
        
"""

In [14]:
rd = Ridge(alpha = 0.02)
rd.fit(x_train,y_train)

y_predict = rd.predict(x_test)
print(rd.coef_)
print("岭回归的均方误差：", mean_squared_error(y_test, y_predict))


[ 0.83166963  0.12159681 -0.26758236  0.30983534 -0.00517992 -0.04040432
 -0.90735215 -0.88211025]
[ 0.83166963  0.12159681 -0.26758236  0.30983534 -0.00517992 -0.04040432
 -0.90735215 -0.88211025]
岭回归的均方误差： 0.5356531179270403


### L1正则化 Lasso回归


In [ ]:
"""
    lasso 回归就是L1正则化，也就是 在普通的损失函数计算中添加一个L = L + λ*|| w ||
    也就是岭回归去掉平方
    这样做可以实现特征选择：当某些特征对目标值的影响较小时，其对应的回归系数会被直接压缩到 0，从而实现 特征选择。
    
    优点：1.特征选择能力：Lasso 回归可以自动将不重要的特征对应的回归系数压缩到 0，从而实现特征选择。
         2.抑制多重共线性：L1 正则化可以限制特征之间的共线性对模型的影响。
         3.简单易用：与普通线性回归相比，Lasso 只增加了一个正则化参数 λ，但增强了模型的泛化能力。
    缺点：1.处理特征多于样本时的不足：如果特征数多于样本数（例如高维数据），Lasso 可能会随机选择部分特征作为非零系数。
         2.则化参数调试成本高：需要通过交叉验证等方法选择最佳 λ
         3.不能很好地处理强相关特征：如果多个特征之间高度相关，Lasso 可能会随机选择其中一个特征而忽略其他特征。
"""

In [16]:
ls = Lasso(alpha = 0.001)

ls.fit(x_train,y_train)
y_predict = ls.predict(x_test)
print(ls.coef_)
print("Lasso回归的均方误差：", mean_squared_error(y_test, y_predict))

[ 0.82655827  0.1225482  -0.25369194  0.29596304 -0.00381001 -0.03948424
 -0.89646842 -0.87060253]
Lasso回归的均方误差： 0.5356324125105497
